In [1]:
import os
import re
import warnings

import pandas as pd
import numpy as np
import torch

from transformers import (
    AutoConfig, AutoTokenizer, AutoModelForSeq2SeqLM, 
    Seq2SeqTrainingArguments, Seq2SeqTrainer, 
    DataCollatorForSeq2Seq, 
)

from datasets import load_metric, Dataset

import wandb
import nltk

os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings('ignore')
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jake/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
NGPU = torch.cuda.device_count()
NCPU = os.cpu_count()
NGPU, NCPU

(1, 16)

# Paths and Names

In [3]:
### paths and names

PROJECT_NAME = 'news-topic-keyphrase-generation-model-dev'
RUN_ID = 'v2'

DATA_PATH = 'data/preprocess_v2.pickle'

MODEL_CHECKPOINT = 'paust/pko-t5-base'
model_name = re.sub(r'[/-]', r'_', MODEL_CHECKPOINT).lower()

METRIC_NAME = 'rouge'

NOTEBOOK_NAME = './train.ipynb'

ROOT_PATH = './'
SAVE_PATH = os.path.join(ROOT_PATH, '.log')

run_name = f'{model_name}_{RUN_ID}'
output_dir = os.path.join(SAVE_PATH, run_name)

print(run_name)
print(output_dir)

!mkdir -p {SAVE_PATH}

paust_pko_t5_base_v2
./.log/paust_pko_t5_base_v2


In [4]:
%env WANDB_PROJECT={PROJECT_NAME}
%env WANDB_NOTEBOOK_NAME={NOTEBOOK_PATH}
%env WANDB_LOG_MODEL=true
%env WANDB_WATCH=all
wandb.login()

env: WANDB_PROJECT=news-topic-keyphrase-generation-model-dev
env: WANDB_NOTEBOOK_NAME={NOTEBOOK_PATH}
env: WANDB_LOG_MODEL=true
env: WANDB_WATCH=all


wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find {NOTEBOOK_PATH}.
wandb: Currently logged in as: dotsnangles. Use `wandb login --relogin` to force relogin


True

# Training Args

In [5]:
report_to="wandb"

num_train_epochs = 15
per_device_train_batch_size = 2
per_device_eval_batch_size = 2
gradient_accumulation_steps = 1

optim = 'adamw_torch' # 'adamw_torch' or 'adamw_hf'

learning_rate = 3e-6 * NGPU
weight_decay = 0.01
adam_epsilon = 1e-8

lr_scheduler_type = 'cosine' # 'linear', 'cosine', 'cosine_with_restarts', 'polynomial', 'constant', 'constant_with_warmup'
warmup_ratio = 0

save_total_limit = 2

load_best_model_at_end = True
metric_for_best_model = 'eval_loss'

save_strategy = "epoch"
evaluation_strategy = "epoch"

logging_strategy = "steps"
logging_first_step = True 
logging_steps = int(500 / NGPU)

predict_with_generate=True
generation_max_length=128
# generation_num_beams=5

fp16 = False

# Model & Tokenizer & Metric

- 모델과 토크나이저, 그리고 평가지표를 계산하는 데 사용할 함수를 불러옵니다.
- 모델의 config에는 사용하지 않는 설정이 포함되어 있습니다. 삭제합니다.

In [6]:
config = AutoConfig.from_pretrained(MODEL_CHECKPOINT)

In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT, config=config)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
metric = load_metric(METRIC_NAME)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Functions

- 모델 훈련을 위해 전처리를 수행해주는 함수를 선언합니다.
- 입력 문장이 input이 되고 요약문이 label이 됩니다.
- tokenizer를 사용해 input_ids로 변환하고 입력값에는 attention_mask를 생성해줍니다.
- 입력 문장에 prefix를 추가하여 성능 실험을 해볼 수 있으나 지금은 적용하지 않겠습니다.

In [8]:
prefix = "generate keyphrases: "

max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["input_text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")

    labels = tokenizer(examples["target_text"], max_length=max_target_length, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

- 모델의 예측값과 위에서 전처리한 라벨을 활용하여 평가지표를 출력하는 함수를 선언합니다.
- ROUGE를 평가지표로 사용합니다.
    - ROUGE-N (N-gram) scoring
    - ROUGE-L (Longest Common Subsequence) scoring
        - sentence-level: Compute longest common subsequence (LCS) between two pieces of text. Newlines are ignored. This is called rougeL in this package.
        - summary-level: Newlines in the text are interpreted as sentence boundaries, and the LCS is computed between each pair of reference and candidate sentences, and something called union-LCS is computed. This is called rougeLsum in this package.

In [9]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

# Inputs and Labels

- csv로 저장된 데이터를 불러와 9:1로 훈련과 검증에 사용합니다.
- 선언한 전처리 함수를 적용하여 데이터세트를 생성합니다.

In [10]:
data_df = pd.read_pickle(DATA_PATH)

In [11]:
dataset = Dataset.from_pandas(data_df).shuffle(seed=100).train_test_split(0.2)
train_dataset = dataset['train']
eval_dataset = dataset['test']

In [12]:
train_dataset = train_dataset.map(preprocess_function, 
                                  batched=True, 
                                  num_proc=NCPU, 
                                  remove_columns=train_dataset.column_names)

eval_dataset = eval_dataset.map(preprocess_function, 
                                batched=True, 
                                num_proc=NCPU, 
                                remove_columns=eval_dataset.column_names)
print(train_dataset)
print(eval_dataset)

Map (num_proc=16):   0%|          | 0/1268 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/317 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1268
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 317
})


In [13]:
tokenizer.decode(train_dataset['input_ids'][0])

'generate keyphrases: 김정섭 공주시장 "유구 채석단지 지정 시 광범위한 피해 우려" 김정섭 공주시장이 7일 정례브리핑을 열고 채석단지 개발에 대해 공식적으로 우려를 표명하고 있다. /공주시 제공 정례브리핑서 "주민 의견 청취해 산림청장에 적극 전달" 김정섭 공주시장이 7일 정례브리핑을 열고 유구읍 채석단지 개발에 대해 공식적으로 우려를 표명했다. 김 시장은 "채석단지 지정 시 발생 가능한 광범위한 피해를 우려한다"며 "주민들의 소중한 의견을 청취해 지정권자인 산림청장에게 적극적으로 전달할 예정"이라고 밝혔다. 이어 "지난 2015년 의당면, 2019년 정안면에서 토석 채취와 관련해 주민들의 강한 반대가 있어 무산된 사실이 있다"며 "생업도 뿌리치고 채석단지 지정 반대를 위해 활동하는 여러 시민들의 입장에서 대응할 것"이라고 덧붙였다. 또 "자연환경을 기반으로 현재까지 시에서 유구읍에 색동수국정원, 유구천 살리기 등의 사업을 추진해 왔다"며 "채석단지 지정이 통과된다면 모든 사업이 퇴색되고 장기적인 환경 피해가 발생할 것으로 예상된다"고 강조했다. 골재 생산 업체는 유구읍 녹천리 관불산 인근에 채석단지 지정을 위한 환경영향평가 초안을 제출하고 지난 5월까지 약 7개월간 주민 의견을 수렴했지만 3300건에 달하는 의견과 함께 2100여명의 탄원서 제출로 반발에 부딪혔다. 주민들은 의견서에서 "석분‧미세먼지 등의 비산으로 반경 2km 이내 인구 밀집지역 각종 피해, 라돈, 석면 등</s>'

In [14]:
tokenizer.decode(train_dataset['labels'][0])

'김정섭 공주시장,유구 채석단지 지정,피해 우려</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

# Train

- 상단에서 선언한 HP와 각종 변수들로 training_args를 생성합니다.

In [15]:
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    run_name=run_name,
    report_to=report_to,

    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,

    optim=optim,

    learning_rate=learning_rate,
    weight_decay=weight_decay,
    adam_epsilon=adam_epsilon,

    lr_scheduler_type=lr_scheduler_type,
    warmup_ratio=warmup_ratio,

    save_total_limit=save_total_limit,

    load_best_model_at_end=load_best_model_at_end,
    metric_for_best_model=metric_for_best_model,

    save_strategy=save_strategy,
    evaluation_strategy=evaluation_strategy,

    logging_strategy=logging_strategy,
    logging_first_step=logging_first_step, 
    logging_steps=logging_steps,

    predict_with_generate=predict_with_generate,
    generation_max_length=generation_max_length,
    # generation_num_beams=generation_num_beams,

    fp16=fp16,
)

- 배치 단위로 padding처리를 하여 훈련 속도를 높히기 위해 data_collator를 선언합니다.
- 생성한 객체들을 trainer에 포함시켜 훈련 준비를 마칩니다.

In [16]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)

trainer = Seq2SeqTrainer(
    model=model,
    
    args=training_args,
    
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    
    tokenizer=tokenizer,
    data_collator=data_collator,
    
    compute_metrics=compute_metrics,
)

- 훈련을 시작합니다.

In [17]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,14.487200,2.447541,3.109300,0.441600,3.064400,3.126000,94.334400
2,2.353100,0.408058,11.426100,1.405700,11.143200,11.179100,24.539400
3,0.642600,0.265997,16.548900,3.740900,15.698300,15.704100,28.375400
4,0.295000,0.239667,18.810700,4.057800,17.933400,18.059500,28.006300
5,0.261800,0.228481,20.204200,4.797200,19.329600,19.399700,27.309100
6,0.246200,0.222539,19.745600,3.958200,18.892200,18.943000,26.817000
7,0.233500,0.218430,20.454700,4.758200,19.498000,19.465700,27.665600
8,0.224600,0.216070,21.004600,4.497300,20.138300,20.113800,26.694000
9,0.221300,0.214330,20.438200,4.061200,19.520400,19.527200,25.845400
10,0.213800,0.212267,21.237600,4.429400,20.394300,20.217500,26.744500


TrainOutput(global_step=9510, training_loss=1.1181987142462586, metrics={'train_runtime': 6085.9148, 'train_samples_per_second': 3.125, 'train_steps_per_second': 1.563, 'total_flos': 1.384213439840256e+16, 'train_loss': 1.1181987142462586, 'epoch': 15.0})

- 옵티마이저 스테이트를 포함한 훈련 재개에 사용할 파일들을 삭제해줍니다.
- 추후 불러올 모델과 토크나이저 관련 파일만 남겨둡니다.

In [18]:
# keep = [
#     'added_tokens.json',
#     'config.json',
#     'pytorch_model.bin',
#     'special_tokens_map.json',
#     'tokenizer.json',
#     'tokenizer_config.json',
#     'vocab.txt'
# ]

# ckpts = os.listdir(output_dir)
# for ckpt in ckpts:
#     ckpt = os.path.join(output_dir, ckpt)
#     for item in os.listdir(ckpt):
#         if item not in keep:
#             os.remove(os.path.join(ckpt, item))

# Generate

- 훈련을 마치면 Evaluation Loss 기준 Best 모델이 로드되어 있습니다.
- trainer는 Greedy Search를 수행하도록 설정되어 있습니다.
- Evaluation 데이터를 활용해 모델의 출력을 간단히 살펴봅니다.

In [19]:
preds = trainer.predict(eval_dataset)

In [20]:
for data, pred in zip(eval_dataset, preds.predictions):
    context = tokenizer.decode(data['input_ids'], skip_special_tokens=True)
    summary = tokenizer.decode(data['labels'], skip_special_tokens=True)
    pred = tokenizer.decode(pred[2:], skip_special_tokens=True)
    # print(f'입력: {context}')
    print(f'정답: {summary}')
    print(f'예측: {pred}', end='\n\n')

정답: 탕화쿵푸마라탕,다온시스템 클린세이버,업무협약 체결,건강한 식음료 문화,코로나 극복,외식업 변화,중국 전통 음식
예측: 화쿵푸마라탕,다온시스템 클린세이버,업무협약 체결,탕화쿵푸마라탕,다온시스템 클린세이버,코로나 시대,건강한 식음료 문화,다온시스템 클린세이버,업무협약 체결

정답: government,protest,colombia,strike
예측: OLOMBIA,PROTESTS,STRIKE

정답: 로봇,오세훈,착공식,4차산업 혁명,로봇인공지능과학관,
예측: I 로봇,오세훈 좌석 안내,착공식 시삽

정답: 츠키,쇼챔 미녀,나나,문수아
예측: 챔,미녀 MC,문수아,나나,츠키,쇼챔

정답: 4연승,전남,안양,리그,전남 드래곤즈,광양 축구전용구장,하나원큐 K리그2
예측: ,안양,리그 1위,리그 4연승,리그 1위,K리그2 2021 9라운드,FC 안양,리그 4연승,리그 1위,전남 드래곤즈

정답: 피아니스트 김수연,2021년 몬트리올 국제 음악 콩쿠르,1위 수상,금호문화재단,금호영재 출신,상금,음반 제작
예측: 문화재단,몬트리올 국제 음악 콩쿠르,피아니스트 김수연,2021년 몬트리올 국제 음악 콩쿠르,피아노 부문

정답: 삼성TV·LG냉장고 더오른다,원자재·물류비 부담,물류난
예측: TV,LG냉장고,원자재,물류비,코로나19,원자재

정답: 공민현,김인균,김민덕,함께가게9월 mvp
예측: 가게,MVP,팬 투표,김민덕,김인균,공민현,대전 하나시티즌,팬 투표

정답: 박지연,시크미,출근길 미모,서울 여의도,KBS,KBS 2TV,금요드라마
예측:  미모,박지연,에이티즈 윤호,데니안,시크미 머금은 출근길 미모

정답: 정진운,경리,이별,웃는 사진,인스타그램,댓글,예의
예측: 운,경리,4년 공개열애 마침표,예의,댓글

정답: 해양안전,해양범죄,구조경진,포항해경,제10회 방제능력 경진,해양치안
예측: 해경,민?관 구조협력 분야,민?관 구조협력 분야,우수파출소,방제능력 경진대회

정답: 아스널 레전드 앙리,맨유 레전드 게리 네빌,브루노 페르난데스,호날두,맨유
예측: ,브루노,비